In [15]:
pip install cdsapi ZipFile

ERROR: Could not find a version that satisfies the requirement ZipFile (from versions: none)
ERROR: No matching distribution found for ZipFile
Note: you may need to restart the kernel to use updated packages.


In [6]:
import cdsapi

c = cdsapi.Client(url = "https://cds.climate.copernicus.eu/api/v2", key = "255794:bedde892-0558-4ee4-b915-54e0fb7d698c")

c.retrieve(
    'sis-temperature-statistics',
    {
        'format': 'zip',
        'variable': 'average_temperature',
        'period': 'year',
        'statistic': 'time_average',
        'experiment': 'rcp4_5',
        'ensemble_statistic': 'ensemble_members_average',
    },
    'data/download.zip')

2023-10-02 13:11:00,829 INFO Welcome to the CDS
2023-10-02 13:11:00,831 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/sis-temperature-statistics
2023-10-02 13:11:00,958 INFO Request is completed
2023-10-02 13:11:00,959 INFO Downloading https://download-0007-clone.copernicus-climate.eu/cache-compute-0007/cache/data3/dataset-sis-temperature-statistics-9bbcd136-11de-44f1-a449-cb4217885d92.zip to data/download.zip (40M)
2023-10-02 13:11:06,114 INFO Download rate 7.8M/s   


Result(content_length=41909241,content_type=application/zip,location=https://download-0007-clone.copernicus-climate.eu/cache-compute-0007/cache/data3/dataset-sis-temperature-statistics-9bbcd136-11de-44f1-a449-cb4217885d92.zip)

In [33]:
import zipfile

def unzip_file(zip_file_path, extract_to_folder):
    try:
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(extract_to_folder)
        print(f"File '{zip_file_path}' has been successfully extracted to '{extract_to_folder}'.")
    except Exception as e:
        print(f"Error: {e}")

# Example usage:
zip_file_path = 'data/download.zip'  # Replace with the actual path to your ZIP file
extract_to_folder = 'data/'         # Replace with the folder where you want to extract the contents

unzip_file(zip_file_path, extract_to_folder)


File 'data/download.zip' has been successfully extracted to 'data/'.
